# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 01: Load, Engineer & Connect</span>

<span style="font-width:bold; font-size: 1.4rem;"> This is the first part of the quick start series of tutorials about Hopsworks Feature Store. As part of this first module, we will work with data related to credit card transactions. 
The objective of this tutorial is to demonstrate how to work with the **Hopworks Feature Store**  for batch data with a goal of training and deploying a model that can predict fraudulent transactions.</span>

## **🗒️ This notebook is divided in 3 sections:** 
1. Loading the data and feature engineeing,
2. Connect to the Hopsworks feature store,
3. Create feature groups and upload them to the feature store.

![tutorial-flow](../images/01_featuregroups.png)


In [1]:
!pip install -U hopsworks~=3.0.0rc3 --quiet

First of all we will load the data and do some feature engineering on it.

## <span style="color:#ff5f27;"> 💽 Loading the Data </span>

The data we will use comes from three different CSV files:

- `credit_cards.csv`: credit card information such as expiration date and provider.
- `transactions.csv`: transaction information such as timestamp, location, and the amount. Importantly, the binary `fraud_label` variable tells us whether a transaction was fraudulent or not.
- `profiles.csv`: credit card user information such as birthdate and city of residence.

We can conceptualize these CSV files as originating from separate data sources.
**All three files have a credit card number column `cc_num` in common, which we can use for joins.**

Let's go ahead and load the data.

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
 
credit_cards_df = pd.read_csv("https://repo.hops.works/master/hopsworks-tutorials/data/card_fraud_online/credit_cards.csv")
credit_cards_df.head(3)

,cc_num,provider,expires
0,4796807885357879,visa,05/23
1,4529266636192966,visa,03/22
2,4922690008243953,visa,02/27


In [2]:
profiles_df = pd.read_csv("https://repo.hops.works/master/hopsworks-tutorials/data/card_fraud_online/profiles.csv", parse_dates=["birthdate"])
profiles_df.head(3)

,name,sex,mail,birthdate,City,Country,cc_num
0,Tonya Gregory,F,sandratorres@hotmail.com,1976-01-16,Far Rockaway,US,4796807885357879
1,Lisa Gilbert,F,michael53@yahoo.com,1986-09-30,Encinitas,US,4529266636192966
2,Carolyn Meyer,F,anthony47@yahoo.com,2001-07-13,Canton,US,4922690008243953


In [3]:
trans_df = pd.read_csv("https://repo.hops.works/master/hopsworks-tutorials/data/card_fraud_online/transactions.csv", parse_dates=["datetime"])
trans_df.head(3)

,tid,datetime,cc_num,category,amount,latitude,longitude,city,country,fraud_label
0,11df919988c134d97bbff2678eb68e22,2022-01-01 00:00:24,4473593503484549,Health/Beauty,62.95,42.30865,-83.48216,Canton,US,0
1,dd0b2d6d4266ccd3bf05bc2ea91cf180,2022-01-01 00:00:56,4272465718946864,Grocery,85.45,33.52253,-117.70755,Laguna Niguel,US,0
2,e627f5d9a9739833bd52d2da51761fc3,2022-01-01 00:02:32,4104216579248948,Domestic Transport,21.63,37.60876,-77.37331,Mechanicsville,US,0


In [4]:
trans_df = trans_df[trans_df.category == "Cash Withdrawal"].reset_index(level=0, drop=True)
trans_df["country"] = trans_df["country"].fillna("US")
profiles_df = profiles_df[profiles_df.cc_num.isin(trans_df.cc_num.unique())].reset_index(level=0, drop=True)
credit_cards_df = credit_cards_df[credit_cards_df.cc_num.isin(trans_df.cc_num.unique())].reset_index(level=0, drop=True)

In [5]:
trans_df.sort_values(["datetime","cc_num"], inplace=True)

## <span style="color:#ff5f27;"> 🛠️ Feature Engineering </span>

Fraudulent transactions can differ from regular ones in many different ways. Typical red flags would for instance be a large transaction volume/frequency in the span of a few hours. It could also be the case that elderly people in particular are targeted by fraudsters. To facilitate model learning we will create additional features based on these patterns. In particular, we will create two types of features:
1. **Features that aggregate data from different data sources**. This could for instance be the age of a customer at the time of a transaction, which combines the `birthdate` feature from `profiles.csv` with the `datetime` feature from `transactions.csv`.
2. **Features that aggregate data from multiple time steps**. An example of this could be the transaction frequency of a credit card in the span of a few hours, which is computed using a window function.

Let's start with the first category.

Next, we create features that for each credit card aggregate data from multiple time steps.

We start by computing the distance between consecutive transactions, which we will call `loc_delta`.
Here we use the [Haversine distance](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.haversine_distances.html?highlight=haversine#sklearn.metrics.pairwise.haversine_distances) to quantify the distance between two longitude and latitude coordinates.

In [6]:
from math import radians

# Feature engineering.
trans_df.sort_values("datetime", inplace=True)
trans_df[["longitude", "latitude"]] = trans_df[["longitude", "latitude"]].applymap(radians)

def haversine(long, lat, shift):
    """Compute Haversine distance between each consecutive coordinate in (long, lat)."""

    long_shifted = long.shift(shift)
    lat_shifted = lat.shift(shift)
    long_diff = long_shifted - long
    lat_diff = lat_shifted - lat

    a = np.sin(lat_diff/2.0)**2
    b = np.cos(lat) * np.cos(lat_shifted) * np.sin(long_diff/2.0)**2
    c = 2*np.arcsin(np.sqrt(a + b))

    return c

def time_delta(datetime_value, shift):
    """Compute time difference between each consecutive transaction."""

    time_shifted = datetime_value.shift(shift)
    return time_shifted

trans_df["loc_delta_t_plus_1"] = trans_df.groupby("cc_num")\
    .apply(lambda x : haversine(x["longitude"], x["latitude"], 1))\
    .reset_index(level=0, drop=True)\
    .fillna(0)

trans_df["loc_delta_t_minus_1"] = trans_df.groupby("cc_num")\
    .apply(lambda x : haversine(x["longitude"], x["latitude"], -1))\
    .reset_index(level=0, drop=True)\
    .fillna(0)

trans_df["time_delta_t_plus_1"] = trans_df.groupby("cc_num")\
    .apply(lambda x : time_delta(x["datetime"], 1 ))\
    .reset_index(level=0, drop=True)

trans_df["time_delta_t_minus_1"] = trans_df.groupby("cc_num")\
    .apply(lambda x : time_delta(x["datetime"],  -1))\
    .reset_index(level=0, drop=True)

trans_df["time_delta_t_plus_1"] = (trans_df.datetime - trans_df.time_delta_t_plus_1 )/ np.timedelta64(1, 'D')
trans_df["time_delta_t_minus_1"] = (trans_df.time_delta_t_minus_1 - trans_df.datetime )/ np.timedelta64(1, 'D')
trans_df["time_delta_t_plus_1"] = trans_df.time_delta_t_plus_1.fillna(0)
trans_df["time_delta_t_minus_1"] = trans_df.time_delta_t_minus_1.fillna(0)  

In [7]:
trans_df = trans_df[["tid","datetime","cc_num","category","amount","city","country","fraud_label","loc_delta_t_plus_1", "loc_delta_t_minus_1", "time_delta_t_plus_1", "time_delta_t_minus_1"]]

In [11]:
trans_df.datetime = trans_df.datetime.values.astype(np.int64) // 10 ** 6
trans_df

,tid,datetime,cc_num,category,amount,city,country,fraud_label,loc_delta_t_plus_1,loc_delta_t_minus_1,time_delta_t_plus_1,time_delta_t_minus_1
169514,4c51b54665c7ddb466ea5936f4f3a428,1641024661000,4467360740682089,Cash Withdrawal,77.77,Norco,US,0,0.000000,0.000148,0.000000,0.333333
167022,4c30185aea2e28e7d9797004710e13c6,1641031422000,4700702588013561,Cash Withdrawal,781.27,Jackson,US,0,0.000000,0.000070,0.000000,0.416667
60585,1a109febabc5c36409f2caf729e110d3,1641031739000,4205094877256105,Cash Withdrawal,36.25,Bothell,US,0,0.000000,0.000108,0.000000,0.333333
249651,855eb0f8e7fdb85e9b63c60028ce1d53,1641033294000,4918033532626920,Cash Withdrawal,93.05,Brighton Beach,US,0,0.000000,0.000032,0.000000,0.250000
203481,b0214ee4d61696adf1149dbaf61fcf2f,1641040494000,4166420904624170,Cash Withdrawal,2.46,Iron River,US,0,0.000000,0.000135,0.000000,0.250000
...,...,...,...,...,...,...,...,...,...,...,...,...
45337,67ce9bd4880f3608bf5ede8e2897e6bb,1651922521000,4795177792394964,Cash Withdrawal,44.23,Middle River,US,0,0.000131,0.000000,6.791667,0.000000
62121,c06a54c7c9832985a527ad17683233e9,1651953137000,4412240992440491,Cash Withdrawal,668.53,Lebanon,US,0,0.000080,0.000000,6.833333,0.000000
183339,3192d04e4e13fc516b3279008ad8a2bf,1651995813000,4600599737277837,Cash Withdrawal,1.79,Morningside Heights,US,0,0.000108,0.000000,6.916667,0.000000
288129,cb4f72270ad50cd0a9b16ff979e7a84c,1652040751000,4382251375646022,Cash Withdrawal,35.64,Glendale,US,0,0.000143,0.000000,6.916667,0.000000


## <span style="color:#ff5f27;"> 🪄 Creating Feature Groups </span>

A [feature group](https://docs.hopsworks.ai/feature-store-api/latest/generated/feature_group/) can be seen as a collection of conceptually related features. In our case, we will create a feature group for the transaction data and a feature group for the windowed aggregations on the transaction data. Both will have `tid` as primary key, which will allow us to join them when creating a dataset in the next tutorial.

Feature groups can also be used to define a namespace for features. For instance, in a real-life setting we would likely want to experiment with different window lengths. In that case, we can create feature groups with identical schema for each window length. 

Before we can create a feature group we need to connect to our feature store.

In [12]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/146
Connected. Call `.close()` to terminate connection gracefully.


To create a feature group we need to give it a name and specify a primary key. It is also good to provide a description of the contents of the feature group and a version number, if it is not defined it will automatically be incremented to `1`.

In [13]:
trans_fg = fs.create_feature_group(
    name="transactions",
    version="1",
    description="Transaction data",
    primary_key=['cc_num'],
    event_time=['datetime'],
    online_enabled=True
)


Here we have also set `online_enabled=True`, which enables low latency access to the data. A full list of arguments can be found in the [documentation](https://docs.hopsworks.ai/feature-store-api/latest/generated/api/feature_store_api/#create_feature_group).

At this point, we have only specified some metadata for the feature group. It does not store any data or even have a schema defined for the data. To make the feature group persistent we populate it with its associated data using the `save` function.

In [14]:
trans_fg.save(trans_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/146/fs/88/fg/67
Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/146/jobs/named/transactions_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7fd763275400>, None)

We can move on and do the same thing for the feature group with our windows aggregation.

Feature group is now accessible and searchable in the UI

![fg-overview](../images/fg_overview.gif)

## <span style="color:#ff5f27;"> 👓  Exploration</span>
In the Hopsworks feature store, the metadata allows for multiple levels of explorations and review. Here we will show a few of those capacities. 

### 🔎 <b>Search</b> 
Using the search function in the ui, you can query any aspect of the feature groups, feature_view and training data that was previously created.

### 📊 <b>Statistics</b> 
We can also enable statistics in the feature groups.

In [ ]:
trans_fg = fs.get_feature_group("transactions", version = 1)
trans_fg.statistics_config = {
    "enabled": True,
    "histograms": True,
    "correlations": True
}

trans_fg.update_statistics_config()
trans_fg.compute_statistics()

![fg-statistics](../images/fg_statistics.gif)


### ⛓️ <b> Lineage </b> 
In all the feature groups and feature view you can look at the relation between each abstractions; what feature group created which training dataset and that is used in which model.
This allows for a clear undestanding of the pipeline in relation to each element. 

## <span style="color:#ff5f27;">⏭️ **Next:** Part 02 </span>

In the following notebook we will use our feature groups to create a dataset we can train a model on.